# Introduction

The notebook is intended to experiment with the Subclassing API of TensorFlow to define custom Neural Network

In [2]:
# Import Standard Libraries
import os

# Suppress warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow import keras

# Linear Layer

In [16]:
class Linear(keras.layers.Layer):
    """
    Define a custom Linear Layer in Keras

    Attributes:
        weights_params: tf.Variables set of weights for each neuron
        bias: tf.Variables set of biases for each neuron
    """
    
    def __init__(self, 
                 units=32):

        # Call the parent constructor             
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):

        # Initilialize the weights
        self.weights_params = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer=tf.random_normal_initializer(),
            trainable=True,
        )

        # Initilialize the bias             
        self.bias = self.add_weight(
            shape=(self.units,),
            initializer=tf.zeros_initializer(),
            trainable=True
        )

    def call(self, inputs):
        
        return tf.matmul(inputs, self.weights_params) + self.bias

In [18]:
# Test
x = tf.ones((2, 2))
linear_layer = Linear(4)
y = linear_layer(x)
print(y)

tf.Tensor(
[[-0.01872915 -0.06430404 -0.0005488  -0.01024083]
 [-0.01872915 -0.06430404 -0.0005488  -0.01024083]], shape=(2, 4), dtype=float32)


In [10]:
# NOTE: The layer has already a "weights" attribute
print(linear_layer.weights == [linear_layer.weights_params, linear_layer.bias])

True


# Multi-layer Perceptron Block

It creates a block layer that combines three custom Linear layers. It tracks the weights of the inner layers.

In [19]:
class MLPBlock(keras.layers.Layer):
    
    def __init__(self):

         # Call the parent constructor
        super(MLPBlock, self).__init__()

        # Define hidden layers
        self.linear_1 = Linear(32)
        self.linear_2 = Linear(32)
        self.linear_3 = Linear(1)

    def call(self, inputs):

        # Pass input to the first hidden layer
        x = self.linear_1(inputs)

        # Trigger the first hidden layer activation function
        x = tf.nn.relu(x)

        # Pass input to the second hidden layer
        x = self.linear_2(x)

        # Trigger the second hidden layer activation function
        x = tf.nn.relu(x)
        
        return self.linear_3(x)

In [22]:
# Test
mlp = MLPBlock()

y = mlp(tf.ones(shape=(3, 64)))  # The first call to the `mlp` will create the weights

print("weights:", len(mlp.weights)) # 3 vector of weights_params and 3 vector of bias
print("trainable weights:", len(mlp.trainable_weights))

weights: 6
trainable weights: 6
